In [3]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [4]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [5]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
#pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
0,1940761,3696391,1
1,1940761,3850720,1
2,1940761,4839681,1
3,1940761,5015362,1
4,1940761,6523930,1
...,...,...,...
585121,11128024,3711313,1
585122,11128039,10224959,1
585123,11128082,4914111,1
585124,11128095,10451987,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [6]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [7]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [8]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [9]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")
        


        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])
        
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [10]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [11]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
3658/3658 [==============================] - 1151s 313ms/step - auc: 0.0000e+00 - rmse: 0.7854 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.8229 - factorized_top_k/top_5_categorical_accuracy: 0.8873 - factorized_top_k/top_10_categorical_accuracy: 0.9120 - factorized_top_k/top_50_categorical_accuracy: 0.9608 - factorized_top_k/top_100_categorical_accuracy: 0.9767 - loss: 616.8134 - regularization_loss: 0.0000e+00 - total_loss: 616.8134
Epoch 2/3
3658/3658 [==============================] - 1144s 313ms/step - auc: 0.0000e+00 - rmse: 10.6435 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 427.2228 - regularization_loss: 0.0000e+00 - total_loss: 427.2228
Epo

In [12]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

3658/3658 [==============================] - 976s 267ms/step - auc: 0.0000e+00 - rmse: 17.6156 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 223.6747 - regularization_loss: 0.0000e+00 - total_loss: 223.6747


([0.0,
  17.615636825561523,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.003333235625177622,
  0,
  0.003333235625177622],
 [1.0, 1.0, 1.0])

In [13]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

1829/1829 [==============================] - 546s 298ms/step - auc: 0.0000e+00 - rmse: 10.2311 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 311.3990 - regularization_loss: 0.0000e+00 - total_loss: 311.3990


([0.0,
  10.231133460998535,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  147.52029418945312,
  0,
  147.52029418945312],
 [1.0, 1.0, 1.0])

In [14]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		10.231133460998535
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		1.0
factorized_top_k/top_5_categorical_accuracy		1.0
factorized_top_k/top_10_categorical_accuracy		1.0
factorized_top_k/top_50_categorical_accuracy		1.0
factorized_top_k/top_100_categorical_accuracy		1.0
AUC: 0.0


In [15]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 6074666,  2103479,  5191659, 10940235,  6613887,  6970448,
        2141843,  2085887,  9369946,  3757095,  4814120,  2235956,
        4931239, 10145631,  9637898,  2129748,  8570856, 10186246,
        9571402,  6259546,  5951032, 10046330, 10863736,  2007368,
       10844649,  8565114,  9737369,  5779155,  2187028,  8694128,
        5834223,  8851065,  8904037,  6056780,  2304042,  7358901,
        7689827,  5516074,  9613902,  9106042, 10656697,  6758887,
        9343055,  9757992,  8567111,  9571330,  9754660,  4495795,
        1988495, 10391194,  8686483,  7335530,  5197667,  5886028,
        6738718,  5859247,  6765228,  9360521,  1964563,  2237602,
        7943600, 11056095, 10885702,  9673399])

In [16]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  6074666
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
916,4770351,4062862813026,Plateau Sandale Glattleder weiß,weiß,64.610.20,6590677,315563,w,SK,F-S,...,5 cm - 8 cm,38.5,5.5,9.0,0.0,PU-TPU,0.0,1.0,"[4770351, 4770358, 4770350, 5507966, 5507961, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
200,3695392,4062862138082,Sneaker low Materialmix Leder/Textil blau,blau,0496.10.01,3692257,3692261,m,PT,H-W,...,bis 3 cm,48.5,13.0,17.0,0.0,TR,0.0,0.0,"[3695392, 3696186, 3695394, 3695378, 3695874, ...","[39.0, 41.0, 42.0, 44.0, 48.5, 49.5]"
314,3849203,4060666972574,Sportliche Ballerina Lederimitat grau,grau,56.471.60,6590678,315571,w,SK,H-W,...,3 cm - 5 cm,41.0,7.5,13.0,0.0,PU-TPU,0.0,1.0,"[3849203, 3849194, 3849200, 3849202, 3849201, ...","[35.5, 36.0, 39.0, 40.0, 40.5, 41.0]"
390,3711195,4062862112303,PG0506 Boot Glattleder braun,braun,0506.51.03,3692257,3703628,m,PT,H-W,...,3 cm - 5 cm,46.0,11.0,13.0,12.0,EVA,0.0,0.0,"[3711195, 3711191, 3694834, 3711192, 3711201, ...","[39.0, 40.0, 41.0, 42.0, 42.5, 43.0, 44.0, 44...."
755,4773901,4064032031780,Schnürschuh Glattleder,,62.065.57,6590678,315577,w,SK,F-S,...,bis 3 cm,39.0,6.0,10.0,0.0,TPU,0.0,1.0,"[4773901, 6142755, 6692674, 7589146, 8964738, ...","[35.5, 37.0, 37.5, 39.0, 40.0, 40.5, 42.5, 43.0]"
975,5338928,4062862753612,Trekkingsandale Glattleder braun,braun,66.829.53,4997827,315596,w,VN,F-S,...,3 cm - 5 cm,40.0,6.5,11.0,0.0,Gummi-EVA,0.0,0.0,"[5338928, 5338053, 5338930, 5338059, 5338057, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1119,4770665,4062862836711,Slingpumps Rauleder Blau,Blau,61.530.16,6590677,315565,w,PT,F-S,...,3 cm - 5 cm,38.5,5.5,9.0,0.0,TPU,0.0,1.0,"[4770665, 7595717, 10444038, 10434628]","[38.5, 42.5, 43.0, 44.0]"
1160,6133383,4064032738504,Sneaker low Rauleder Grau,Grau,73.361.19,6590677,315567,w,SK,H-W,...,3 cm - 5 cm,41.0,7.5,13.0,0.0,TR,0.0,0.0,"[6133383, 6109997, 6116422, 7790897, 7790893, ...","[37.0, 38.0, 38.5, 39.0, 40.0, 41.0, 42.0, 43...."
1399,6460557,4064032466292,Sneaker low Materialmix Leder/Lederimitat Beige,Beige,76.528.39,6590678,315567,w,SK,H-W,...,3 cm - 5 cm,41.0,7.5,13.0,0.0,TR,0.0,0.0,"[6460557, 6460553, 6460554, 6460555]","[38.5, 39.0, 40.0, 41.0]"
1857,7790844,4064032264164,Slipper Rauleder beige,beige,84.194.12,6590677,315567,w,PT,F-S,...,3 cm - 5 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[7790844, 7790846, 7790847, 7790848, 7790845, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
2620,8487873,4065171228635,Eleganter Pumps Materialmix Leder schwarz,schwarz,91.461.37,6590677,315593,w,PT,H-W,...,5 cm - 8 cm,38.0,5.0,8.0,0.0,TPU,0.0,0.0,"[8487873, 8487876, 8487877, 8487879, 8487875, ...","[35.0, 37.0, 38.0, 39.0, 40.0, 40.5, 42.0]"


scores:  [0.16039291 0.13147944 0.12098805 0.11940132 0.11908958 0.11087607
 0.10865597 0.10801742 0.10274546 0.10252981]


In [32]:
#tf.saved_model.save(model, '../../models/gabor/gabor_tensorflow_reco_all_data_no_metadata_3_epochs_v1')

model.save_weights('../../models/gabor2/gabor_tensorflow_reco_all_data_no_metadata_3_epochs_v1_weights', save_format='tf')
